In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from pathlib import Path
import time
import random

# Check if we are running on a CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We need to install the transformers library first, if we use Google Colab.

In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 901kB 55.5MB/s 
     |████████████████████████████████| 3.3MB 54.2MB/s 


From transformers, we need only the model itself, which is `DistilBertForSequenceClassification`; special optimizer that works with it, which is `AdamW`; and a pretrained tokenizer `DistilBertTokenizer` to feed our data correctly into the model. 

For the sake of saving space and performance, we use `DistilBert` in this notebook.

In [3]:
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig
from transformers import DistilBertTokenizer
from transformers import get_linear_schedule_with_warmup
!kaggle competitions download -c shopee-product-matching!

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


# Downloading and unpacking the data.

In [4]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
titles = pd.read_csv("/content/drive/My Drive/NN/train.csv")
titles_only = titles["title"]
label_grps = titles["label_group"] 
titles

In [ ]:
df = pd.DataFrame()
t1 = []
t2 = []
same = []
c = 0
for i in range(titles.shape[0]):
  for j in range(titles.shape[0]):
    t1.append(titles_only[i])
    t2.append(titles_only[j])
    t = label_grps[i]==label_grps[j]
    c += t
    same.append(t)
  if i % 100 == 0:
    print(i, c)
  if c == 10000:
    break
df["t1"] = t1
df["t2"] = t2
df["same"] = same


In [47]:
df = pd.DataFrame()
df["t1"] = t1[:-1]
df["t2"] = t2[:-1]
df["same"] = same
df.to_csv("/content/drive/My Drive/NN/trainform.csv")

In [48]:
 df = pd.read_csv("/content/drive/My Drive/NN/trainform.csv")

In [50]:
tr = df.loc[df["same"] == True]
tr
fl = df.loc[df["same"] == False].reset_index()
fl
perc = np.random.rand(fl.shape[0])
perc = perc < (tr.shape[0]*5/fl.shape[0])
fl = fl[perc]
fl.to_csv("/content/drive/My Drive/NN/train_tr.csv")
tr.to_csv("/content/drive/My Drive/NN/train_fl.csv")

In [5]:
tr = pd.read_csv("/content/drive/My Drive/NN/train_fl.csv")
fl = pd.read_csv("/content/drive/My Drive/NN/train_tr.csv")
perc = np.random.rand(fl.shape[0])
perc = perc < (tr.shape[0]/fl.shape[0])
fl = fl[perc]

In [56]:
fl

,Unnamed: 0,index,Unnamed: 0.1,t1,t2,same
8,19695,19696,19696,Paper Bag Victoria Secret,Sajadah Praktis Souvenir Siraman Unik Abu Dhab...,False
13,23024,23025,23025,Paper Bag Victoria Secret,Overloose Basic Top Sourcesoriginal,False
14,24168,24169,24169,Paper Bag Victoria Secret,(BISA COD) Paket Lengkap Filter Air Zernii Wat...,False
21,33171,33173,33173,Paper Bag Victoria Secret,Bedong bayi instan katun halus praktis resleti...,False
30,50399,50402,50402,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",DIGITAL WATCH NINOCS ( BERGARANSI ),False
...,...,...,...,...,...,...
68258,77825815,77839370,77839370,RAK SUDUT Aluminium SA317 / RAK TOILET TEMPAT ...,GROSIR DRESS ANAK KAYSA - DASTER ARAB ANAK - F...,False
68259,77826089,77839645,77839645,RAK SUDUT Aluminium SA317 / RAK TOILET TEMPAT ...,ASAH PISAU SURMENE ASAHAN PISAU MURAH ASAHAN T...,False
68262,77827700,77841256,77841256,RAK SUDUT Aluminium SA317 / RAK TOILET TEMPAT ...,Pengaman laci lemari pintu perabotan dari bayi...,False
68266,77831842,77845398,77845398,RAK SUDUT Aluminium SA317 / RAK TOILET TEMPAT ...,Mainan Puzzle Edukasi Anak,False


In [14]:
dataa = {}
#data = [fl,tr]
labs = {}
for i in [0,1]:
  #p = data[i]
  #perc = np.random.rand(p.shape[0])
  #perc = perc > 0.8
  #p["test"] = perc
  #dataa[i] = p
  labs[i] = i

Initializig the tokenizer from the pretrained `distilbert-base-uncased` model.

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case=True, padding_side="right")

In [8]:
print(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today."))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today.")))
tokenizer.encode_plus("This is the BERT tokenizer that we're going to use today.")

['this', 'is', 'the', 'bert', 'token', '##izer', 'that', 'we', "'", 're', 'going', 'to', 'use', 'today', '.']
[2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012]


{'input_ids': [101, 2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
PAD = '[PAD]'
PAD_ID = 0

batch_size = 16
validation_split = .3
shuffle_dataset = True
random_seed = 42

Here, we are going to re-use the `IMDBDataSet` class that we defined in the Lab 4. The only difference is that we get rid of pretrained word embeddings and replace a Spacy tokenizer with our pretrained `DistilBertTokenizer`.

Remember, that in order to use BERT model, our sequence must have the `[CLS]` special token in the beginning and `[SEP]` in the end of the sequence. If we are using `DistilBertTokenizer.encode()` method, we can just specify `add_special_tokens=True` to not bother with it manually.

Another important thing is that the pretrained `distilbert-base-uncased` model only accepts sequences of maximum length of 512 tokens. We set the `max_length` parameter to the `DistilBertTokenizer.encode()` method to trim them accordingly.

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

Since we already return torch tensors padded to have equal lengths in the Dataset, we don't need to define a custom `collate_fn` function as we did in the Lab 4. The default `collate_fn` function from `DataLoader` will handle the collating for us.

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

In [10]:
class IMDBDataSet(Dataset):
    def __init__(self, folders, pretrain_tokenizer, data, test=False):
        self.tokenizer = pretrain_tokenizer
        self.label_vocab = folders
        self.max_len = 512
        self.sign = False
        self.in_data= data

        if test:
            self.sign = True
        else:
            self.sign = False
            
        self.data = []
        
        self.load()
        
    def load(self):
        for label in self.label_vocab:
          p = self.in_data[self.label_vocab[label]].dropna()
          p = p.loc[p["test"]==self.sign]
          print(f'Reading {label} sentences...')
          for t1,t2 in zip(p["t1"],p["t2"]):
            text = self.tokenizer.encode(t1,t2, add_special_tokens = True, max_length=self.max_len, padding= "max_length", truncation = True, return_tensors = "pt")
            attention_mask = text > 0
            attention_mask = attention_mask.squeeze()
            torch_label = torch.tensor(self.label_vocab[label], dtype = torch.long)
              # append text, attention text and torch_label
            self.data.append((text.squeeze(), attention_mask, torch_label))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1], self.data[idx][2]

In [11]:
train_data = IMDBDataSet(folders = labs,pretrain_tokenizer=tokenizer, data=dataa)

Reading 0 sentences...
Reading 1 sentences...


In [12]:
test_data = IMDBDataSet(folders = labs,pretrain_tokenizer=tokenizer, data=dataa, test=True)

Reading 0 sentences...
Reading 1 sentences...


In [ ]:
train_data[0]

In [14]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [15]:
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels= len(train_data.label_vocab), output_attentions = False, output_hidden_states=False)
model.to(device)

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The DistilBERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [7]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

epochs = 1
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

NameError: ignored

In [8]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [23]:
def multi_label_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    confusion_vector = rounded_preds / y
    
    true_positives = torch.sum(confusion_vector == 1,dtype=torch.float)
    false_positives = torch.sum(torch.isinf(confusion_vector),dtype=torch.float)
    false_negatives = torch.sum(confusion_vector == 0 ,dtype=torch.float)
    true_negatives = torch.sum(torch.isnan(confusion_vector),dtype=torch.float)

    accuracy = (true_positives+true_negatives) / (true_positives+true_negatives+false_negatives+false_positives)
    precision = (true_positives) / (true_positives+false_positives)
    recall = (true_positives) / (true_positives+false_negatives)
    f_score = 2*(precision*recall)/(precision+recall)
    return accuracy, precision, recall, f_score

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [24]:
# Taken from this tutorial: https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb
# The code was modified

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed_mins, elapsed_secs = epoch_time(t0, time.time())
            
            # Report progress.
            print(f'  Batch {step:>5,}  of  {len(train_loader):>5,}.    Elapsed: {elapsed_mins:}m {elapsed_secs:}s.')

        # Unpack this training batch from our dataloader. 
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, attention_mask= b_input_mask, labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print(f"  Average training loss: {avg_train_loss:.4f}")
    print("  Training epcoh took: {:}m {:}s".format(*epoch_time(t0, time.time())))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
            outputs = model(b_input_ids, attention_mask= b_input_mask)


        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()
        #print(label_ids)
        #print( np.argmax(logits, axis=1).flatten())
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.6f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}m {:}s".format(*epoch_time(t0, time.time())))

print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...
  Batch    40  of    949.    Elapsed: 0m 29s.
  Batch    80  of    949.    Elapsed: 0m 59s.
  Batch   120  of    949.    Elapsed: 1m 32s.
  Batch   160  of    949.    Elapsed: 2m 4s.
  Batch   200  of    949.    Elapsed: 2m 37s.
  Batch   240  of    949.    Elapsed: 3m 9s.
  Batch   280  of    949.    Elapsed: 3m 42s.
  Batch   320  of    949.    Elapsed: 4m 14s.
  Batch   360  of    949.    Elapsed: 4m 47s.
  Batch   400  of    949.    Elapsed: 5m 19s.
  Batch   440  of    949.    Elapsed: 5m 52s.
  Batch   480  of    949.    Elapsed: 6m 24s.
  Batch   520  of    949.    Elapsed: 6m 57s.
  Batch   560  of    949.    Elapsed: 7m 29s.
  Batch   600  of    949.    Elapsed: 8m 2s.
  Batch   640  of    949.    Elapsed: 8m 34s.
  Batch   680  of    949.    Elapsed: 9m 7s.
  Batch   720  of    949.    Elapsed: 9m 39s.
  Batch   760  of    949.    Elapsed: 10m 12s.
  Batch   800  of    949.    Elapsed: 10m 44s.
  Batch   840  of    949.    Elapsed: 1

In [25]:
print("")
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0

# Evaluate data for one epoch
for batch in test_loader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, b_input_mask)


    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to("cpu").numpy()
    print(label_ids)
    print( np.argmax(logits, axis=1).flatten())
    # Calculate the accuracy for this batch of test sentences.
    tmp_test_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    test_accuracy += tmp_test_accuracy

    # Track the number of batches
    nb_test_steps += 1

# Report the final accuracy for this test run.
print("  Accuracy: {0:.2f}".format(test_accuracy/nb_test_steps))
print("  Testing took: {:}m {:}s".format(*epoch_time(t0, time.time())))


Running Testing...
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [11]:
def predict_same(model, s1, s2, voc):
    model.eval()
    text = tokenizer.encode(s1,s2, add_special_tokens = True, max_length=512, padding= "max_length", truncation = True, return_tensors = "pt")
    attention_mask = text > 0
    attention_mask = attention_mask.squeeze().to(device)
    text = text.to(device)
    bind = (text, attention_mask)
    #print(text, attention_mask)
    with torch.no_grad():
      outputs = model(bind[0], bind[1])[0]
    proba = outputs.detach().cpu().numpy()
    prediction, indices = torch.topk(torch.sigmoid(outputs), 1)
    indices = indices.detach().cpu().numpy()
    mojis = [voc[pred]for pred in indices[0]]
    return mojis, proba[0]


In [18]:
print(predict_same(model, "Victoria secret underware","Victoria secret watch", labs))

([1], array([-2.611601 ,  3.3308225], dtype=float32))


In [19]:
train = pd.read_csv("/content/drive/My Drive/NN/train.csv")
test = pd.read_csv("/content/drive/My Drive/NN/test.csv")

In [ ]:
post_ids = []
matches = []
buf = []
for i in range(test.shape[0]):
    post_ids.append(test.loc[i]['posting_id'])
    buf = str(test.loc[i]['posting_id'])
    for j in range(train.shape[0]):
        res = predict_same(model, test.loc[i]['title'],train.loc[j]['title'], labs)[0][0]
        if j % 10000 == 0:
            print(j)
        if res == 1:
            buf +=  " " + str(train.loc[j]['posting_id'])
        
    matches.append(buf)

In [27]:
sub = pd.DataFrame()
sub["posting_id"] = post_ids
sub["matches"] = matches



In [26]:

import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/My Drive/NN/model/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/drive/My Drive/NN/model/


('/content/drive/My Drive/NN/model/tokenizer_config.json',
 '/content/drive/My Drive/NN/model/special_tokens_map.json',
 '/content/drive/My Drive/NN/model/vocab.txt',
 '/content/drive/My Drive/NN/model/added_tokens.json')

In [10]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = '/content/drive/My Drive/NN/model/'
model = DistilBertForSequenceClassification.from_pretrained(output_dir)
tokenizer = DistilBertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       